In [468]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import pycountry_convert

In [469]:
pd.options.display.max_seq_items = 10

 Read the file.

In [470]:
df = pd.read_parquet('Result_file_compressed.parquet')
df.head()

,Date:,Time:,Location:,Operator:,Flight #:,Route:,AC Type:,Registration:,cn / ln:,Aboard:,Fatalities:,Ground:,Summary:
0,"January 15, 2023",1050,"Pokhara, Nepal",Yeti Airlines,YT691,Kathmandu - Pokhara,ATR 72-500-72-212-A,9NANC,754,72 (passengers:68 crew:4),72 (passengers:68 crew:4),0,"Before approach, the pilot requested a change ..."
1,"November 21, 2022",1015,"Medellín, Colombia",AeroPaca SAS,?,Medellín - Pizarro,Piper PA-31-350 Navajo Chieftain,HK-5121,31-7652004,8 (passengers:6 crew:2),8 (passengers:6 crew:2),0,The plane was chartered to carry a team of six...
2,"November 18, 2022",1511,"Lima, Peru",LATAM,LA2213,Lima - Juliaca,Airbus 320-271N,CC-BHB,7864,108 (passengers:102 crew:6),0 (passengers:0 crew:0),2,The Airbus A320 collided with a fire truck whi...
3,"November 06, 2022",0853,"Bukoba, Tanzania",Precision Air,PW494,Dar es-Salaam -Bukoba,ATR 42-500,5H-PWF,819,43 (passengers:39 crew:39),19 (passengers:17 crew:2),0,"While on final approach to Bukoba Airport, the..."
4,"July 16, 2022",2247,"Eleftheroupolis, Greece",Meridian,MEM3032,Nis- Amman,Antonov An-12,UR-CIC,01347701,8 (passengers:0 crew:8),8 (passengers:0 crew:8),0,The cargo plane carrying eight crew members an...


In [471]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4596 entries, 0 to 4595
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Date:          4596 non-null   object
 1   Time:          4596 non-null   object
 2   Location:      4596 non-null   object
 3   Operator:      4596 non-null   object
 4   Flight #:      4596 non-null   object
 5   Route:         4596 non-null   object
 6   AC  Type:      4596 non-null   object
 7   Registration:  4596 non-null   object
 8   cn / ln:       4596 non-null   object
 9   Aboard:        4596 non-null   object
 10  Fatalities:    4596 non-null   object
 11  Ground:        4596 non-null   object
 12  Summary:       4596 non-null   object
dtypes: object(13)
memory usage: 466.9+ KB


Drop the summary column.

In [472]:
df = df.drop('Summary:', axis=1)

Remove the colons from each of the column names and remove the spaces

In [473]:
df.columns = df.columns.str[:-1]
replacement_dict = {'  ': '_', ' ': '_',  '/': '', '#': 'No', '__': '_'}

for original, replacement in replacement_dict.items():
    df.columns = df.columns.str.replace(original, replacement)
    
df.columns

Index(['Date', 'Time', 'Location', 'Operator', 'Flight_No',
       ...
       'Registration', 'cn_ln', 'Aboard', 'Fatalities', 'Ground'],
      dtype='object', length=12)

Performing some data type conversions

In [474]:
df['Date'] = pd.to_datetime(df['Date'])
df['Date'].dtypes


dtype('<M8[ns]')

In the time format, let's remove the colons and other anomalies so that we can parse them later on.

In [475]:
symbols_to_replace = [':', ';', 'c: ', 'c:', 'Z', 'c ', 'c']
replacement_mapping = {symbol: '' for symbol in symbols_to_replace}

for symbol, replacement in replacement_mapping.items():
    df['Time'] = df['Time'].str.replace(symbol, replacement)

Introduce a new 'decade' column.

In [476]:
df['Decade'] = df['Date'].dt.year // 10 * 10

Extracting the country name from the location column

Replace some exceptions.

In [477]:
df['Location'] = df['Location'].replace({
    'Minsk, Belarus, USSR': 'Minsk, Belarus',
    'Southern Belarus, USSR': 'Southern Belarus, Belarus'
})

In [478]:
location_parts = df['Location'].str.split(',')

df['City'] = location_parts.apply(lambda x: ', '.join(x[:-1]))
df['Country/State/Other'] = location_parts.apply(lambda x: x[-1].strip())


In [479]:
values = df['Country/State/Other'].unique()
values.sort()
values

array(['110 miles West of Ireland', '175 miles off the Egyptian coast',
       '325 miles east of Wake Island', '800 miles east of Newfoundland',
       '?', 'AK', 'Aegean Sea', 'Afghanistan', 'Afghanstan', 'Africa',
       'Airzona', 'Alabama', 'Alaksa', 'Alaska', 'Albania', 'Algeria',
       'Algiers', 'American Samoa', 'Amsterdam', 'Andes', 'Angola',
       'Antarctica', 'Antigua', 'Aregntina', 'Argentina', 'Arizona',
       'Arkansas', 'Armenia', 'Atlantic Ocean',
       'Atlantic Ocean between N.Y. and Bermuda',
       'Atlantic Ocean off Florida', 'Australia', 'Australila', 'Austria',
       'Azerbaijan', 'Azores', 'Azores (Portugal)', 'BO', 'Baangladesh',
       'Bahamas', 'Bahrain', 'Baltic Sea', 'Bangladesh', 'Barbados',
       'Barquisimeto Venezuela', 'Belarus', 'Belgian Congo',
       'Belgian Congo (Zaire)', 'Belgium', 'Belgium Congo', 'Benin',
       'Bermuda', 'Bhutan', 'Black Sea', 'Boliva', 'Bolivia', 'Borneo',
       'Bosnia', 'Bosnia-Herzegovina', 'Botswana', 'Brazil

So now a problem is that US states are used, and the name of the country is not.

In [480]:
df_states = pd.read_html('https://en.wikipedia.org/wiki/List_of_states_and_territories_of_the_United_States')

In [481]:
df_states = df_states[1]

In [482]:
state_names = df_states.iloc[:, 0].values

In [483]:
state_names

array(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
       'Colorado', 'Connecticut', 'Delaware', 'Florida', 'Georgia',
       'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas',
       'Kentucky[B]', 'Louisiana', 'Maine', 'Maryland',
       'Massachusetts[B]', 'Michigan', 'Minnesota', 'Mississippi',
       'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire',
       'New Jersey', 'New Mexico', 'New York', 'North Carolina',
       'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania[B]',
       'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee',
       'Texas', 'Utah', 'Vermont', 'Virginia[B]', 'Washington',
       'West Virginia', 'Wisconsin', 'Wyoming'], dtype=object)

There are some names which contain [B]. These are now replaced.

In [484]:
state_names = [name.replace('[B]', '') for name in state_names]

There are some spelling mistakes, which are now corrected.

In [485]:
area_values = df['Country/State/Other'].unique()
area_values.sort()
for value in area_values:
    print(value)

110 miles West of Ireland
175 miles off the Egyptian coast
325 miles east of Wake Island
800 miles east of Newfoundland
?
AK
Aegean Sea
Afghanistan
Afghanstan
Africa
Airzona
Alabama
Alaksa
Alaska
Albania
Algeria
Algiers
American Samoa
Amsterdam
Andes
Angola
Antarctica
Antigua
Aregntina
Argentina
Arizona
Arkansas
Armenia
Atlantic Ocean
Atlantic Ocean between N.Y. and Bermuda
Atlantic Ocean off Florida
Australia
Australila
Austria
Azerbaijan
Azores
Azores (Portugal)
BO
Baangladesh
Bahamas
Bahrain
Baltic Sea
Bangladesh
Barbados
Barquisimeto Venezuela
Belarus
Belgian Congo
Belgian Congo (Zaire)
Belgium
Belgium Congo
Benin
Bermuda
Bhutan
Black Sea
Boliva
Bolivia
Borneo
Bosnia
Bosnia-Herzegovina
Botswana
Brazil
British Cameroons
British Columbia Canada
British Virgin Islands
Brunei
Bulgaria
Bulgeria
Burma
Burma (Myanmar)
CA
Cailifornia
California
Californiia
Calilfornia
Cambodia
Cameroon
Cameroons
Canada
Canada2
Canary Islands
Cape Verde
Cape Verde Islands
Central African Republic
Central Mo

In [486]:
df.query("`Country/State/Other` == 'BO'")

,Date,Time,Location,Operator,Flight_No,Route,AC_Type,Registration,cn_ln,Aboard,Fatalities,Ground,Decade,City,Country/State/Other
662,2000-12-11,1445,"Near Sumacal, Beni, BO",Servicio Aereo Manchupo,?,Cashaschanes - Sumacal,Cessna 206,CP-1721,?,6 (passengers:5 crew:1),6 (passengers:5 crew:1),0,2000,"Near Sumacal, Beni",BO


In [487]:
df['Country/State/Other'] = df['Country/State/Other'].replace({
    'AK': 'Alaska',
    'Afghanstan': 'Afghanistan',
    'Airzona': 'Arizona',
    'Alaksa': 'Alaska',
    'Aregntina': 'Argentina',
    'Australila': 'Australia',
    'Baangladesh': 'Bangladesh',
    'Azores (Portugal)': 'Azores',
    'Belgian Congo (Zaire)': 'Belgian Congo',
    'Belgium Congo': 'Belgian Congo',
    'BO': 'Bolivia',
    'Boliva': 'Bolivia',
    'Bulgeria': 'Bulgaria',
    'Burma': 'Myanmar',
    'Burma (Myanmar)': 'Myanmar',
    'CA': 'California',
    'Cailifornia': 'California',
    'Californiia': 'California',
    'Calilfornia': 'California',
    'Cameroons': 'Cameroon',
    'Canada2': 'Canada',
    'Coatia': 'Croatia',
    'Coloado': 'Colorado',
    'Colombia': 'Columbia',
    'Comoro Islands': 'Comoros Islands',
    'Comoros': 'Comoros Islands',
    'D.C.': 'Washington',
    'Deleware': 'Delaware',
    'Democratic Republic Cogo': 'Democratic Republic of the Congo',
    'Democratic Republic Congo': 'Democratic Republic of the Congo',
    'Democratic Republic of Congo': 'Democratic Republic of the Congo',
    'Democtratic Republic Congo': 'Democratic Republic of the Congo',
    'Djbouti': 'Djibouti',
    'Domincan Republic': 'Dominican Republic',
    'Forest-in-Teesdale. United Kingdom': 'United Kingdom',
    'HI': 'Hawaii',
    'HI)': 'Hawaii',
    'Ilinois': 'Illinois',
    'India.': 'India',
    'Indian': 'India',
    'Inodnesia': 'Indonesia',
    'Jamacia': 'Jamaica',
    'Kent': 'United Kingdom',
    'Malta International Airport': 'Malta',
    'Massachutes': 'Massachusetts',
    'Mauretania': 'Mauritania',
    'Mexic': 'Mexico',
    'Minnisota': 'Minnesota',
    'Mississipi': 'Mississippi',
    'Morroco': 'Morocco',
    'Napal': 'Nepal',
    'New York (Idlewild)': 'New York',
    'Papua': 'Papua New Guinea',
    'Philippines': 'Phillipines',
    'Republic of Djibouti': 'Djibouti',
    'Republic of Georgia': 'Georgia',
    'Qld. Australia': 'Australia',
    'Quebec Canada': 'Canada',
    'Queensland Australia': 'Australia',
    'Russian': 'Russia',
    'Saskatchewan': 'Canada',
    'Saudia Arabia': 'Saudi Arabia',
    'South Dekota': 'South Dakota',
    'Surinam': 'Suriname',
    'Swden': 'Sweden',
    'Taiwan (Formosa)': 'Taiwan',
    'Tennesee': 'Tennessee',
    'The Netherlands': 'Netherlands',
    'Unied Kingdom': 'United Kingdom',
    'WY': 'Wyoming',
    'Washingon': 'Washington',
    'Wisconson': 'Wisconsin',
    'Yugosalvia': 'Yugoslavia',
    'Zimbabwe)': 'Zimbabwe',
    'bulgaria': 'Bulgaria',
    'UK': 'United Kingdom',
    'England': 'United Kingdom',
    
    # DOing the rest
    '110 miles West of Ireland': 'Ireland',
    '175 miles off the Egyptian coast': 'Egypt',
    '325 miles east of Wake Island': 'Pacific Ocean',
    'Amsterdam': 'Netherlands',
    'Atlantic Ocean between N.Y. and Bermuda': 'Atlantic Ocean',
    'Atlantic Ocean off Florida': 'Atlantic Ocean',
    'Azores': 'Atlantic Ocean',
    'Barquisimeto Venezuela': 'Venezuela',
    'Chechnya': 'Russia',
    'Congo Democratic Republic': 'Democratic Republic of the Congo',
    'Crete': 'Greece',
    'DR Congo': 'Democratic Republic of Congo',
    'Dutch Guyana': 'Guyana',
    'East Germany': 'Germany',
    'East Pakistan': 'Pakistan',
    'East Pakistan (Bangladesh)': 'Bangladesh',
    'East Sardinia': 'Italy',
    'East Libya': 'Libya',
    'En route from Argentina to California': 'Argentina',
    'English Channel': 'United Kingdom',
    'Fox Glacier Airstrip': 'New Zealand',
    'Great Inagua': 'Bahamas',
    'Guizhou Province': 'China',
    'Gulf of Finland': 'Finland',
    'Gulf of Mexico': 'Mexico',
    'Gulf of Oman': 'Oman',
    'Gulf of Thailand': 'Thailand',
    'Hati': 'Haiti',
    'Hunary': 'Hungary',
    'Ilha Grande Rio de Janeiro': 'Brazil',
    'Islay Island': 'United Kingdom',
    'Kirghizia': 'Kyrgyzstan',
    'Manitoba': 'Canada',
    'Manmar': 'Myanmar',
    'Midway Island Naval Air Station': 'United States of America',
    'Milford Sound': 'New Zealand',
    'Mt. Helmos. Greece': 'Greece',
    'NE of Bermuda': 'Bermuda',
    'NY': 'New York',
    'Near Hong Kong': 'Hong Kong',
    'Near Hong Kong International Airport': 'Hong Kong',
    'Near Houma Louisiana': 'Louisiana',
    'Near Irkutsk Russia': 'Russia',
    'Near Jacquinot Bay New Guinea': 'Papua New Guinea',
    'Near Karkov': 'Ukraine',
    'Near Lete Pass': 'Nepal',
    'Near Petreasa Romania': 'Romania',
    'Near Tachikawa Air Base': 'Japan',
    'Near Villia Greece': 'Greece',
    'Newfoundland': 'Canada',
    'North Atlantic': 'Atlantic Ocean',
    'North Atlantic Ocean': 'Atlantic Ocean',
    'North Pacific Ocean': 'Pacific Ocean',
    'Northeast Laos': 'Laos',
    'Northern Afghanistan': 'Afghanistan',
    'Northern Iraq': 'Iraq',
    'Northern Ireland': 'Ireland',
    'Northern Israel': 'Israel',
    'Northwest Territories Canada': 'Canada',
    'Czechoslovakia': 'Czech Republic',
    'Bosnia-Herzegovina': 'Bosnia and Herzegovina',
    'Bosnia': 'Bosnia and Herzegovina',
    'Algiers': 'Algeria'
})

If the country/state/other column contains one of the US state names,show it as USA.

Also preserve the state name in another column

In [488]:
temp = np.where(df['Country/State/Other'].isin(state_names), 'United States of America', df['Country/State/Other'])
df['Country_new'] = temp
df['US_State'] = np.where(df['Country/State/Other'].isin(state_names), df['Country/State/Other'], None)

Now replace the country

In [489]:
df = df.drop('Country/State/Other', axis=1)
df = df.rename(columns={'Country_new': 'Country'})
df = df.drop('Location', axis=1)

In [490]:
df.head()

,Date,Time,Operator,Flight_No,Route,AC_Type,Registration,cn_ln,Aboard,Fatalities,Ground,Decade,City,Country,US_State
0,2023-01-15,1050,Yeti Airlines,YT691,Kathmandu - Pokhara,ATR 72-500-72-212-A,9NANC,754,72 (passengers:68 crew:4),72 (passengers:68 crew:4),0,2020,Pokhara,Nepal,None
1,2022-11-21,1015,AeroPaca SAS,?,Medellín - Pizarro,Piper PA-31-350 Navajo Chieftain,HK-5121,31-7652004,8 (passengers:6 crew:2),8 (passengers:6 crew:2),0,2020,Medellín,Columbia,None
2,2022-11-18,1511,LATAM,LA2213,Lima - Juliaca,Airbus 320-271N,CC-BHB,7864,108 (passengers:102 crew:6),0 (passengers:0 crew:0),2,2020,Lima,Peru,None
3,2022-11-06,0853,Precision Air,PW494,Dar es-Salaam -Bukoba,ATR 42-500,5H-PWF,819,43 (passengers:39 crew:39),19 (passengers:17 crew:2),0,2020,Bukoba,Tanzania,None
4,2022-07-16,2247,Meridian,MEM3032,Nis- Amman,Antonov An-12,UR-CIC,01347701,8 (passengers:0 crew:8),8 (passengers:0 crew:8),0,2020,Eleftheroupolis,Greece,None


Now, there are values of 'Country' which contains stuff like 'Over', 'Off', 'Near', etc. Which I will now remove.

In [491]:
prefix_regex = r'^(Near|Off the|off the|Over the|Off|off|Over|the)\s'

df['Country'] = df['Country'].str.replace(prefix_regex, '', regex=True)

Replacing the rest of the values

In [492]:
df['Country'] = df['Country'].replace({
    'Alaska coast': 'United States of America',
    'Bimini': 'Bahamas',
    'Cape Verde Islands': 'Cape Verde',
    'Chili': 'Chile',
    'Eastern Libya': 'Libya',
    'English Channel': 'United Kingdom',
    'Eugene Island': 'United States of America',
    'Gulf of Finland': 'Finland',
    'Gulf of Karkinitsky': 'Ukraine',
    'Gulf of Sirte': 'Libya',
    'Gulf of Tonkin': 'Vietnam',
    'Irish coast': 'Ireland',
    'Malta-Luqa': 'Malta',
    'Mediterranean': 'Mediterranean Sea',
    'North Atlantic': 'Atlantic Ocean',
    'North Pacific Ocean': 'Pacific Ocean',
    'Northern Germany': 'Germany',
    'Okinawa': 'Japan',
    'Ontario': 'Canada',
    'Oregon coast': 'United States of America',
    'Pacific Ocean between Hong Kong and Macao': 'Pacific Ocean',
    'Pacific Ocean between Manila and Guam': 'Pacific Ocean',
    'Panama coast': 'Panama',
    'Phillipines': 'Philippines',
    'Philipines': 'Philippines',
    'Philippine Sea': 'Philippines',
    'Philippine island of Elalat': 'Philipines',
    'Rhodesia (Zimbabwe)': 'Zimbabwe',
    'Santiago de Cuba': 'Cuba',
    'Sao Gabriel de Cachoeria': 'Brazil',
    'Sarawak': 'Malaysia',
    'Scotland': 'United Kingdom',
    'Sea of Japan': 'Japan',
    'Sierre Leone': 'Sierra Leone',
    'South Atlantic Ocean': 'Atlantic Ocean',
    'South Australia': 'Australia',
    'South Indian Ocean': 'Indian Ocean',
    'South Kazakistan': 'Kazakistan',
    'South Vietnam': 'Vietnam',
    'South Yemen': 'Yemen',
    'South of Gibraltar': 'Gibraltar',
    'Southeastern Bolivia': 'Bolivia',
    'South-West Africa (Namibia)': 'Namibia',
    'Soviet Union': 'Russia',
    'Spain Canary Islands': 'Spain',
    'Tabones Island Philippines': 'Philippines',
    'Tanganyika': 'Tanzania',
    'Tasmania': 'Australia',
    'USSR': 'Russia',
    'U.S. Virgin Islands': 'US Virgin Islands',
    'UAE': 'United Arab Emirates',
    'UAR': 'Egypt',
    'United States': 'United States of America',
    'Uzbekstan': 'Uzbekistan',
    'Virgin Islands': 'US Virgin Islands',
    'Washington': 'United States of America',
    'West Germany': 'Germany',
    'West Pakistan': 'Pakistan',
    'Western Samoa': 'Samoa',
    'Yukon Territory': 'Canada',
    'coast of France': 'France',
    'western Denmark': 'Denmark',
    '800 miles east of Newfoundland': 'Canada',
    'of Gibraltar': 'Gibraltar',
    'South Koren': 'South Korea',
    'Pest Hungary': 'Hungary',
    'British Columbia Canada': 'Canada',
    'Columbia': 'Colombia',
    'Washington D.C.': 'United States of America',
    'French Somaliland': 'Somalia',
    'South Korean': 'South Korea',
    'Papua New Guinea': 'New Guinea',
    'Philipines': 'Philippines',
    'Territory of New Guinea': 'New Guinea',
    'Upper Volta': 'Burkina Faso',
    'of Gibraltar': 'Gibraltar',
    'Zaire': 'Democratic Republic of the Congo',
    'Kazakistan': 'Kazakhstan',
    'Central Mozambique': 'Mozambique',
    '?': 'Unknown',
    'Algiers': 'Algeria'
})

In [493]:
area_values = df['Country'].unique()
area_values.sort()
for value in area_values:
    print(value)

Aegean Sea
Afghanistan
Africa
Albania
Algeria
American Samoa
Andaman Sea
Andes
Angola
Antarctica
Antigua
Argentina
Armenia
Atlantic Ocean
Australia
Austria
Azerbaijan
Azores
Bahamas
Bahrain
Baltic Sea
Bangladesh
Barbados
Belarus
Belgian Congo
Belgium
Benin
Bermuda
Bhutan
Black Sea
Bolivia
Borneo
Bosnia and Herzegovina
Botswana
Brazil
British Cameroons
British Virgin Islands
Brunei
Bulgaria
Burkina Faso
Cambodia
Cameroon
Canada
Canary Islands
Cape Verde
Carribean Sea
Central African Republic
Chad
Chile
China
Colombia
Comoros Islands
Congo
Cook Islands
Costa Rica
Croatia
Cuba
Cyprus
Czech Republic
Democratic Republic of Congo
Democratic Republic of the Congo
Denmark
Desertores Island Región de Los Lagos
Djibouti
Dominican Republic
East Timor
Ecuador
Egypt
El Salvador
Equatorial Guinea
Eritrea
Estonia
Ethiopia
Fiji
Finland
France
French Cameroons
French Equatorial Africa
French Indo-China
French Polynesia
French West Africa
French West Indies
Gabon
Gambia
Germany
Ghana
Gibraltar
Greece
Gr

### Finding number of people on board, fatalities and survivors.

In [494]:
import re

In [495]:
test_str = df.head(1)['Aboard'].values.tolist()[0]
test_str

'72 (passengers:68 crew:4)'

In [496]:
passenger_parser = re.compile(
    '''
    (\d+)               # Total
    \s                  # Blank space
    \(                  # Opening bracket
    passengers:         # Fixed string
    (.+)                # Number of passengers
    \s
    crew:
    (.+)                # Number of crew
    \)
    ''', re.VERBOSE)

Processing the number of people abroad

In [497]:
abroad_parts = df['Aboard'].str.extract(passenger_parser)
df['Total_abroad'], df['Passengers_abroad'], df['Crew_abroad'] = abroad_parts[0], abroad_parts[1], abroad_parts[2]

Processing the number of casualties

In [498]:
fatalities_parts = df['Fatalities'].str.extract(passenger_parser)
df['Total_fatalities'], df['Passengers_fatalities'], df['Crew_fatalities'] = fatalities_parts[0], fatalities_parts[1], fatalities_parts[2]

Now that the column has been split into three, the original columns can be dropped.

In [499]:
df = df.drop(['Fatalities', 'Aboard'], axis=1)
df.head()

,Date,Time,Operator,Flight_No,Route,AC_Type,Registration,cn_ln,Ground,Decade,City,Country,US_State,Total_abroad,Passengers_abroad,Crew_abroad,Total_fatalities,Passengers_fatalities,Crew_fatalities
0,2023-01-15,1050,Yeti Airlines,YT691,Kathmandu - Pokhara,ATR 72-500-72-212-A,9NANC,754,0,2020,Pokhara,Nepal,None,72,68,4,72,68,4
1,2022-11-21,1015,AeroPaca SAS,?,Medellín - Pizarro,Piper PA-31-350 Navajo Chieftain,HK-5121,31-7652004,0,2020,Medellín,Colombia,None,8,6,2,8,6,2
2,2022-11-18,1511,LATAM,LA2213,Lima - Juliaca,Airbus 320-271N,CC-BHB,7864,2,2020,Lima,Peru,None,108,102,6,0,0,0
3,2022-11-06,0853,Precision Air,PW494,Dar es-Salaam -Bukoba,ATR 42-500,5H-PWF,819,0,2020,Bukoba,Tanzania,None,43,39,39,19,17,2
4,2022-07-16,2247,Meridian,MEM3032,Nis- Amman,Antonov An-12,UR-CIC,01347701,0,2020,Eleftheroupolis,Greece,None,8,0,8,8,0,8


### Splitting route into source and destination.

In [500]:
route_parts = df['Route'].str.split('-')

destination = route_parts.apply(lambda x: x[-1])
source = route_parts.apply(lambda x: '-'.join(x[:-1]))

df['Source'] = source
df['Destination'] = destination

# Now strip away the trailing whitespaces
df['Source'] = df['Source'].str.strip()
df['Destination'] = df['Destination'].str.strip()

Now drop the route.

In [501]:
df = df.drop('Route', axis=1)

Now convert some other columns

In [502]:
df['Time'] = pd.to_datetime(df['Time'], format='%H%M', errors='coerce').dt.time

In [503]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4596 entries, 0 to 4595
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Date                   4596 non-null   datetime64[ns]
 1   Time                   3223 non-null   object        
 2   Operator               4596 non-null   object        
 3   Flight_No              4596 non-null   object        
 4   AC_Type                4596 non-null   object        
 5   Registration           4596 non-null   object        
 6   cn_ln                  4596 non-null   object        
 7   Ground                 4596 non-null   object        
 8   Decade                 4596 non-null   int64         
 9   City                   4596 non-null   object        
 10  Country                4596 non-null   object        
 11  US_State               964 non-null    object        
 12  Total_abroad           4582 non-null   object        
 13  Pas

In [504]:
num_cols = ['Total_abroad', 'Ground', 'Passengers_abroad', 'Crew_abroad', 'Total_fatalities', 'Passengers_fatalities', 'Crew_fatalities']

for column in num_cols:
    df[column] = df[column].replace({'?': np.nan})
    df[column] = df[column].astype('Int16')

In [505]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4596 entries, 0 to 4595
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Date                   4596 non-null   datetime64[ns]
 1   Time                   3223 non-null   object        
 2   Operator               4596 non-null   object        
 3   Flight_No              4596 non-null   object        
 4   AC_Type                4596 non-null   object        
 5   Registration           4596 non-null   object        
 6   cn_ln                  4596 non-null   object        
 7   Ground                 4556 non-null   Int16         
 8   Decade                 4596 non-null   int64         
 9   City                   4596 non-null   object        
 10  Country                4596 non-null   object        
 11  US_State               964 non-null    object        
 12  Total_abroad           4582 non-null   Int16         
 13  Pas

Introduce three new survivors colmns.

In [506]:
df['Total_survivors'] = df['Total_abroad'] - df['Total_fatalities']
df['Passengers_survivors'] = df['Passengers_abroad'] - df['Passengers_fatalities']
df['Crew_survivors'] = df['Crew_abroad'] - df['Crew_fatalities']

In [507]:
place_names = sorted(df['Destination'].unique().tolist())
with open('Place_names.txt', 'w') as file:
    for name in place_names:
        file.write(f'{name}\n')

In [508]:
month_order = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
day_order = ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']

In [509]:
pd.set_option('display.max_columns', 50)

## 1.1. Make a column for survival percentage

In [510]:
df['Survival_rate'] = (df['Total_survivors'] / df['Total_abroad']) * 100
df.head()

,Date,Time,Operator,Flight_No,AC_Type,Registration,cn_ln,Ground,Decade,City,Country,US_State,Total_abroad,Passengers_abroad,Crew_abroad,Total_fatalities,Passengers_fatalities,Crew_fatalities,Source,Destination,Total_survivors,Passengers_survivors,Crew_survivors,Survival_rate
0,2023-01-15,10:50:00,Yeti Airlines,YT691,ATR 72-500-72-212-A,9NANC,754,0,2020,Pokhara,Nepal,None,72,68,4,72,68,4,Kathmandu,Pokhara,0,0,0,0.0
1,2022-11-21,10:15:00,AeroPaca SAS,?,Piper PA-31-350 Navajo Chieftain,HK-5121,31-7652004,0,2020,Medellín,Colombia,None,8,6,2,8,6,2,Medellín,Pizarro,0,0,0,0.0
2,2022-11-18,15:11:00,LATAM,LA2213,Airbus 320-271N,CC-BHB,7864,2,2020,Lima,Peru,None,108,102,6,0,0,0,Lima,Juliaca,108,102,6,100.0
3,2022-11-06,08:53:00,Precision Air,PW494,ATR 42-500,5H-PWF,819,0,2020,Bukoba,Tanzania,None,43,39,39,19,17,2,Dar es-Salaam,Bukoba,24,22,37,55.813953
4,2022-07-16,22:47:00,Meridian,MEM3032,Antonov An-12,UR-CIC,01347701,0,2020,Eleftheroupolis,Greece,None,8,0,8,8,0,8,Nis,Amman,0,0,0,0.0


## 1.2 Making a type column for making passenger or military aircraft

In [511]:
def determine_type(value):
    if 'military' in value.lower():
        return 'Military'
    else:
        return 'Passenger'
    
df['Type'] = df['Operator'].apply(lambda x: determine_type(x))
df.head()

,Date,Time,Operator,Flight_No,AC_Type,Registration,cn_ln,Ground,Decade,City,Country,US_State,Total_abroad,Passengers_abroad,Crew_abroad,Total_fatalities,Passengers_fatalities,Crew_fatalities,Source,Destination,Total_survivors,Passengers_survivors,Crew_survivors,Survival_rate,Type
0,2023-01-15,10:50:00,Yeti Airlines,YT691,ATR 72-500-72-212-A,9NANC,754,0,2020,Pokhara,Nepal,None,72,68,4,72,68,4,Kathmandu,Pokhara,0,0,0,0.0,Passenger
1,2022-11-21,10:15:00,AeroPaca SAS,?,Piper PA-31-350 Navajo Chieftain,HK-5121,31-7652004,0,2020,Medellín,Colombia,None,8,6,2,8,6,2,Medellín,Pizarro,0,0,0,0.0,Passenger
2,2022-11-18,15:11:00,LATAM,LA2213,Airbus 320-271N,CC-BHB,7864,2,2020,Lima,Peru,None,108,102,6,0,0,0,Lima,Juliaca,108,102,6,100.0,Passenger
3,2022-11-06,08:53:00,Precision Air,PW494,ATR 42-500,5H-PWF,819,0,2020,Bukoba,Tanzania,None,43,39,39,19,17,2,Dar es-Salaam,Bukoba,24,22,37,55.813953,Passenger
4,2022-07-16,22:47:00,Meridian,MEM3032,Antonov An-12,UR-CIC,01347701,0,2020,Eleftheroupolis,Greece,None,8,0,8,8,0,8,Nis,Amman,0,0,0,0.0,Passenger


## 1.3 Making columns for day of the week and month name

In [512]:
df['Month'] = df['Date'].dt.month
df['Month'] = df['Month'].apply(lambda x: pd.datetime.strptime(str(x), '%m').strftime('%B'))
df['Day_of_week'] = df['Date'].dt.strftime('%A')
df.iloc[:5, -2:]

C:\Users\user\AppData\Local\Temp\ipykernel_11680\29041736.py:2: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  df['Month'] = df['Month'].apply(lambda x: pd.datetime.strptime(str(x), '%m').strftime('%B'))


,Month,Day_of_week
0,January,Sunday
1,November,Monday
2,November,Friday
3,November,Sunday
4,July,Saturday


In [513]:
df['Month'] = pd.Categorical(df['Month'], categories=month_order, ordered=True)
df['Day_of_week'] = pd.Categorical(df['Day_of_week'], categories=day_order, ordered=True)
df['US_State'] = pd.Categorical(df['US_State'])

In [514]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4596 entries, 0 to 4595
Data columns (total 27 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Date                   4596 non-null   datetime64[ns]
 1   Time                   3223 non-null   object        
 2   Operator               4596 non-null   object        
 3   Flight_No              4596 non-null   object        
 4   AC_Type                4596 non-null   object        
 5   Registration           4596 non-null   object        
 6   cn_ln                  4596 non-null   object        
 7   Ground                 4556 non-null   Int16         
 8   Decade                 4596 non-null   int64         
 9   City                   4596 non-null   object        
 10  Country                4596 non-null   object        
 11  US_State               964 non-null    category      
 12  Total_abroad           4582 non-null   Int16         
 13  Pas

## 1.4 Continent column

In [515]:
continent_dict = {'NA': 'North America', 'SA': 'South America', 'EU': 'Europe', 'AF': 'Africa', 'AS': 'Asia', 'OC': 'Oceania'}

In [516]:
def get_continent_code(x):
    try:
        country_code = pycountry_convert.country_name_to_country_alpha2(x)
        continent_code = pycountry_convert.country_alpha2_to_continent_code(country_code)
        return continent_code
    except:
        return None

df['Continent Code'] = df['Country'].apply(lambda x: get_continent_code(x))
df['Continent'] = df['Continent Code'].map(continent_dict)
df = df.drop('Continent Code', axis=1)

# SAVE THE DATAFRAME

In [517]:
df.to_csv('Processed_dataset/Crash_data_new.csv', index=False)
df.to_parquet('Processed_dataset/Crash_data_new.parquet')

In [518]:
df['US_State'].unique()

[NaN, 'California', 'Alaska', 'Ohio', 'Texas', ..., 'Oregon', 'Rhode Island', 'Nebraska', 'Oklahoma', 'Delaware']
Length: 50
Categories (49, object): ['Alabama', 'Alaska', 'Arizona', 'Arkansas', ..., 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming']

In [519]:
df.head()

,Date,Time,Operator,Flight_No,AC_Type,Registration,cn_ln,Ground,Decade,City,Country,US_State,Total_abroad,Passengers_abroad,Crew_abroad,Total_fatalities,Passengers_fatalities,Crew_fatalities,Source,Destination,Total_survivors,Passengers_survivors,Crew_survivors,Survival_rate,Type,Month,Day_of_week,Continent
0,2023-01-15,10:50:00,Yeti Airlines,YT691,ATR 72-500-72-212-A,9NANC,754,0,2020,Pokhara,Nepal,NaN,72,68,4,72,68,4,Kathmandu,Pokhara,0,0,0,0.0,Passenger,January,Sunday,Asia
1,2022-11-21,10:15:00,AeroPaca SAS,?,Piper PA-31-350 Navajo Chieftain,HK-5121,31-7652004,0,2020,Medellín,Colombia,NaN,8,6,2,8,6,2,Medellín,Pizarro,0,0,0,0.0,Passenger,November,Monday,South America
2,2022-11-18,15:11:00,LATAM,LA2213,Airbus 320-271N,CC-BHB,7864,2,2020,Lima,Peru,NaN,108,102,6,0,0,0,Lima,Juliaca,108,102,6,100.0,Passenger,November,Friday,South America
3,2022-11-06,08:53:00,Precision Air,PW494,ATR 42-500,5H-PWF,819,0,2020,Bukoba,Tanzania,NaN,43,39,39,19,17,2,Dar es-Salaam,Bukoba,24,22,37,55.813953,Passenger,November,Sunday,Africa
4,2022-07-16,22:47:00,Meridian,MEM3032,Antonov An-12,UR-CIC,01347701,0,2020,Eleftheroupolis,Greece,NaN,8,0,8,8,0,8,Nis,Amman,0,0,0,0.0,Passenger,July,Saturday,Europe


In [520]:
area_values = df['Country'].unique()
area_values.sort()
for value in area_values:
    print(value)

Aegean Sea
Afghanistan
Africa
Albania
Algeria
American Samoa
Andaman Sea
Andes
Angola
Antarctica
Antigua
Argentina
Armenia
Atlantic Ocean
Australia
Austria
Azerbaijan
Azores
Bahamas
Bahrain
Baltic Sea
Bangladesh
Barbados
Belarus
Belgian Congo
Belgium
Benin
Bermuda
Bhutan
Black Sea
Bolivia
Borneo
Bosnia and Herzegovina
Botswana
Brazil
British Cameroons
British Virgin Islands
Brunei
Bulgaria
Burkina Faso
Cambodia
Cameroon
Canada
Canary Islands
Cape Verde
Carribean Sea
Central African Republic
Chad
Chile
China
Colombia
Comoros Islands
Congo
Cook Islands
Costa Rica
Croatia
Cuba
Cyprus
Czech Republic
Democratic Republic of Congo
Democratic Republic of the Congo
Denmark
Desertores Island Región de Los Lagos
Djibouti
Dominican Republic
East Timor
Ecuador
Egypt
El Salvador
Equatorial Guinea
Eritrea
Estonia
Ethiopia
Fiji
Finland
France
French Cameroons
French Equatorial Africa
French Indo-China
French Polynesia
French West Africa
French West Indies
Gabon
Gambia
Germany
Ghana
Gibraltar
Greece
Gr

In [521]:
sorted(df['Country'].unique())

['Aegean Sea',
 'Afghanistan',
 'Africa',
 'Albania',
 'Algeria',
 'American Samoa',
 'Andaman Sea',
 'Andes',
 'Angola',
 'Antarctica',
 'Antigua',
 'Argentina',
 'Armenia',
 'Atlantic Ocean',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Azores',
 'Bahamas',
 'Bahrain',
 'Baltic Sea',
 'Bangladesh',
 'Barbados',
 'Belarus',
 'Belgian Congo',
 'Belgium',
 'Benin',
 'Bermuda',
 'Bhutan',
 'Black Sea',
 'Bolivia',
 'Borneo',
 'Bosnia and Herzegovina',
 'Botswana',
 'Brazil',
 'British Cameroons',
 'British Virgin Islands',
 'Brunei',
 'Bulgaria',
 'Burkina Faso',
 'Cambodia',
 'Cameroon',
 'Canada',
 'Canary Islands',
 'Cape Verde',
 'Carribean Sea',
 'Central African Republic',
 'Chad',
 'Chile',
 'China',
 'Colombia',
 'Comoros Islands',
 'Congo',
 'Cook Islands',
 'Costa Rica',
 'Croatia',
 'Cuba',
 'Cyprus',
 'Czech Republic',
 'Democratic Republic of Congo',
 'Democratic Republic of the Congo',
 'Denmark',
 'Desertores Island Región de Los Lagos',
 'Djibouti',
 'Dominican Republic',
 